In [1]:
import onnx
from tvm.contrib.download import download_testdata
from PIL import Image
import numpy as np
import tvm.relay as relay
import tvm
from tvm.contrib import graph_executor

In [8]:
model_url = (
    "https://github.com/onnx/models/raw/main/"
    "vision/classification/resnet/model/"
    "resnet50-v2-7.onnx"
)

model_path = download_testdata(model_url, "resnet50-v2-7.onnx", module="onnx")
onnx_model = onnx.load(model_path)

# Seed numpy's RNG to get consistent results
np.random.seed(0)

In [9]:
img_url = "https://s3.amazonaws.com/model-server/inputs/kitten.jpg"
img_path = download_testdata(img_url, "imagenet_cat.png", module="data")

# Resize it to 224x224
resized_image = Image.open(img_path).resize((224, 224))
img_data = np.asarray(resized_image).astype("float32")

# Our input image is in HWC layout while ONNX expects CHW input, so convert the array
img_data = np.transpose(img_data, (2, 0, 1))

# Normalize according to the ImageNet input specification
imagenet_mean = np.array([0.485, 0.456, 0.406]).reshape((3, 1, 1))
imagenet_stddev = np.array([0.229, 0.224, 0.225]).reshape((3, 1, 1))
norm_img_data = (img_data / 255 - imagenet_mean) / imagenet_stddev

# Add the batch dimension, as we are expecting 4-dimensional input: NCHW.
img_data = np.expand_dims(norm_img_data, axis=0)

In [10]:
target = "llvm"

In [11]:
# The input name may vary across model types. You can use a tool
# like Netron to check input names
input_name = "data"
shape_dict = {input_name: img_data.shape}

mod, params = relay.frontend.from_onnx(onnx_model, shape_dict)

with tvm.transform.PassContext(opt_level=3):
    lib = relay.build(mod, target=target, params=params)

dev = tvm.device(str(target), 0)
module = graph_executor.GraphModule(lib["default"](dev))

/home/wendell/Desktop/tvm/python/tvm/driver/build_module.py:267: UserWarning: target_host parameter is going to be deprecated. Please pass in tvm.target.Target(target, host=target_host) instead.
  warnings.warn(
[14:46:55] /home/wendell/Desktop/tvm/src/runtime/threading_backend.cc:320: Warning: more than two frequencies detected!
One or more operators have not been tuned. Please tune your model for better performance. Use DEBUG logging level to see more details.


In [12]:
dtype = "float32"
module.set_input(input_name, img_data)
module.run()
output_shape = (1, 1000)
tvm_output = module.get_output(0, tvm.nd.empty(output_shape)).numpy()

In [13]:
import timeit

timing_number = 10
timing_repeat = 10
unoptimized = (
    np.array(timeit.Timer(lambda: module.run()).repeat(repeat=timing_repeat, number=timing_number))
    * 1000
    / timing_number
)
unoptimized = {
    "mean": np.mean(unoptimized),
    "median": np.median(unoptimized),
    "std": np.std(unoptimized),
}

print(unoptimized)

{'mean': 104.74954335018992, 'median': 104.9895281001227, 'std': 0.8132510949467782}


In [14]:
from scipy.special import softmax

# Download a list of labels
labels_url = "https://s3.amazonaws.com/onnx-model-zoo/synset.txt"
labels_path = download_testdata(labels_url, "synset.txt", module="data")

with open(labels_path, "r") as f:
    labels = [l.rstrip() for l in f]

# Open the output and read the output tensor
scores = softmax(tvm_output)
scores = np.squeeze(scores)
ranks = np.argsort(scores)[::-1]
for rank in ranks[0:5]:
    print("class='%s' with probability=%f" % (labels[rank], scores[rank]))

class='n02123045 tabby, tabby cat' with probability=0.621103
class='n02123159 tiger cat' with probability=0.356379
class='n02124075 Egyptian cat' with probability=0.019712
class='n02129604 tiger, Panthera tigris' with probability=0.001215
class='n04040759 radiator' with probability=0.000262


In [15]:
import tvm.auto_scheduler as auto_scheduler
from tvm.autotvm.tuner import XGBTuner
from tvm import autotvm

In [16]:
number = 10
repeat = 1
min_repeat_ms = 0  # since we're tuning on a CPU, can be set to 0
timeout = 10  # in seconds

# create a TVM runner
runner = autotvm.LocalRunner(
    number=number,
    repeat=repeat,
    timeout=timeout,
    min_repeat_ms=min_repeat_ms,
    enable_cpu_cache_flush=True,
)

In [17]:
tuning_option = {
    "tuner": "xgb",
    "trials": 20,
    "early_stopping": 100,
    "measure_option": autotvm.measure_option(
        builder=autotvm.LocalBuilder(build_func="default"), runner=runner
    ),
    "tuning_records": "resnet-50-v2-autotuning.json",
}

In [18]:
tasks = autotvm.task.extract_from_program(mod["main"], target=target, params=params)

# Tune the extracted tasks sequentially.
for i, task in enumerate(tasks):
    prefix = "[Task %2d/%2d] " % (i + 1, len(tasks))
    tuner_obj = XGBTuner(task, loss_type="rank")
    tuner_obj.tune(
        n_trial=min(tuning_option["trials"], len(task.config_space)),
        early_stopping=tuning_option["early_stopping"],
        measure_option=tuning_option["measure_option"],
        callbacks=[
            autotvm.callback.progress_bar(tuning_option["trials"], prefix=prefix),
            autotvm.callback.log_to_file(tuning_option["tuning_records"]),
        ],
    )

[15:01:55] /home/wendell/Desktop/tvm/src/runtime/threading_backend.cc:320: Warning: more than two frequencies detected!


[Task  1/25]  Current/Best:    0.00/   0.00 GFLOPS | Progress: (0/20) | 0.00 s

[15:02:15] /home/wendell/Desktop/tvm/src/runtime/threading_backend.cc:320: Warning: more than two frequencies detected!
[15:02:15] /home/wendell/Desktop/tvm/src/runtime/threading_backend.cc:320: Warning: more than two frequencies detected!
[15:02:15] /home/wendell/Desktop/tvm/src/runtime/threading_backend.cc:320: Warning: more than two frequencies detected!
[15:02:16] /home/wendell/Desktop/tvm/src/runtime/threading_backend.cc:320: Warning: more than two frequencies detected!
[15:02:16] /home/wendell/Desktop/tvm/src/runtime/threading_backend.cc:320: Warning: more than two frequencies detected!
[15:02:16] /home/wendell/Desktop/tvm/src/runtime/threading_backend.cc:320: Warning: more than two frequencies detected!
[15:02:16] /home/wendell/Desktop/tvm/src/runtime/threading_backend.cc:320: Warning: more than two frequencies detected!
[15:02:16] /home/wendell/Desktop/tvm/src/runtime/threading_backend.cc:320: Warning: more than two frequencies detected!
[15:02:17] /home/wendell/Desktop/tvm/src

[Task  1/25]  Current/Best:   32.87/ 105.40 GFLOPS | Progress: (20/20) | 8.44 s

[15:02:19] /home/wendell/Desktop/tvm/src/runtime/threading_backend.cc:320: Warning: more than two frequencies detected!


...

[Task 23/25]  Current/Best:   28.27/ 102.01 GFLOPS | Progress: (20/20) | 8.44 s Done.
[Task 24/25]  Current/Best:    0.00/   0.00 GFLOPS | Progress: (0/20) | 0.00 s

[15:05:18] /home/wendell/Desktop/tvm/src/runtime/threading_backend.cc:320: Warning: more than two frequencies detected!
[15:05:18] /home/wendell/Desktop/tvm/src/runtime/threading_backend.cc:320: Warning: more than two frequencies detected!
[15:05:18] /home/wendell/Desktop/tvm/src/runtime/threading_backend.cc:320: Warning: more than two frequencies detected!
[15:05:18] /home/wendell/Desktop/tvm/src/runtime/threading_backend.cc:320: Warning: more than two frequencies detected!
[15:05:19] /home/wendell/Desktop/tvm/src/runtime/threading_backend.cc:320: Warning: more than two frequencies detected!
[15:05:19] /home/wendell/Desktop/tvm/src/runtime/threading_backend.cc:320: Warning: more than two frequencies detected!
[15:05:19] /home/wendell/Desktop/tvm/src/runtime/threading_backend.cc:320: Warning: more than two frequencies detected!
[15:05:19] /home/wendell/Desktop/tvm/src/runtime/threading_backend.cc:320: Warning: more than two frequencies detected!
[15:05:19] /home/wendell/Desktop/tvm/src

[Task 25/25]  Current/Best:    0.00/   0.00 GFLOPS | Progress: (0/20) | 0.00 s s Done.


[15:05:32] /home/wendell/Desktop/tvm/src/runtime/threading_backend.cc:320: Warning: more than two frequencies detected!
[15:05:32] /home/wendell/Desktop/tvm/src/runtime/threading_backend.cc:320: Warning: more than two frequencies detected!
[15:05:32] /home/wendell/Desktop/tvm/src/runtime/threading_backend.cc:320: Warning: more than two frequencies detected!
[15:05:33] /home/wendell/Desktop/tvm/src/runtime/threading_backend.cc:320: Warning: more than two frequencies detected!
[15:05:33] /home/wendell/Desktop/tvm/src/runtime/threading_backend.cc:320: Warning: more than two frequencies detected!
[15:05:33] /home/wendell/Desktop/tvm/src/runtime/threading_backend.cc:320: Warning: more than two frequencies detected!
[15:05:34] /home/wendell/Desktop/tvm/src/runtime/threading_backend.cc:320: Warning: more than two frequencies detected!
[15:05:34] /home/wendell/Desktop/tvm/src/runtime/threading_backend.cc:320: Warning: more than two frequencies detected!
[15:05:34] /home/wendell/Desktop/tvm/src

[Task 25/25]  Current/Best:    9.10/  38.00 GFLOPS | Progress: (20/20) | 13.51 s

In [19]:
with autotvm.apply_history_best(tuning_option["tuning_records"]):
    with tvm.transform.PassContext(opt_level=3, config={}):
        lib = relay.build(mod, target=target, params=params)

dev = tvm.device(str(target), 0)
module = graph_executor.GraphModule(lib["default"](dev))

 Done.


/home/wendell/Desktop/tvm/python/tvm/driver/build_module.py:267: UserWarning: target_host parameter is going to be deprecated. Please pass in tvm.target.Target(target, host=target_host) instead.
  warnings.warn(


In [20]:
dtype = "float32"
module.set_input(input_name, img_data)
module.run()
output_shape = (1, 1000)
tvm_output = module.get_output(0, tvm.nd.empty(output_shape)).numpy()

scores = softmax(tvm_output)
scores = np.squeeze(scores)
ranks = np.argsort(scores)[::-1]
for rank in ranks[0:5]:
    print("class='%s' with probability=%f" % (labels[rank], scores[rank]))

class='n02123045 tabby, tabby cat' with probability=0.621104
class='n02123159 tiger cat' with probability=0.356378
class='n02124075 Egyptian cat' with probability=0.019712
class='n02129604 tiger, Panthera tigris' with probability=0.001215
class='n04040759 radiator' with probability=0.000262


In [21]:
import timeit

timing_number = 10
timing_repeat = 10
optimized = (
    np.array(timeit.Timer(lambda: module.run()).repeat(repeat=timing_repeat, number=timing_number))
    * 1000
    / timing_number
)
optimized = {"mean": np.mean(optimized), "median": np.median(optimized), "std": np.std(optimized)}


print("optimized: %s" % (optimized))
print("unoptimized: %s" % (unoptimized))

optimized: {'mean': 88.02220112993382, 'median': 87.82353450078517, 'std': 1.0479282204824454}
unoptimized: {'mean': 104.74954335018992, 'median': 104.9895281001227, 'std': 0.8132510949467782}
